<a href="https://colab.research.google.com/github/moonstar0301/PairsTrading_PaperReadingGroup/blob/main/%EC%BD%94%EC%8A%A4%ED%94%BC%20%EC%A0%84%EC%A2%85%EB%AA%A9%EC%97%90%EC%84%9C%EC%9D%98%20%ED%8E%98%EC%96%B4%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%94%A9%20%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/sources/data_paristrading.xlsx", index_col=0)

df.index.name = 'Date'

df

In [ ]:
df.iloc[:2]

In [ ]:
import os
import glob
import math
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

'''Import data'''

class Constants: # object to store constants we need
    def __init__(self, start_train, end_train, start_test, end_test):
        self.start_train = start_train
        self.end_train = end_train
        self.start_test = start_test
        self.end_test = end_test

def normalize_data(df, column_name):
    df[column_name] = (df[column_name] - df[column_name].mean()) / df[column_name].std()

def get_paths(fpath, extension=""):
    '''
    returns dictionary of {stock_name:path_to_csv}
    '''
    all_files = glob.glob(fpath + f"/*{extension}") # get all paths in directory
    return {os.path.basename(v):v for v in all_files}

def dt_parse(s):
    return dt.datetime.strptime(s, "%Y-%m-%d")

import pandas as pd
import numpy as np
from dateutil.parser import parse as dt_parse  # dt_parse 함수를 가져옵니다.

def get_dataframes(constants):
    start_train = dt_parse(constants.start_train)
    end_train = dt_parse(constants.end_train)
    end_test = dt_parse(constants.end_test)

    df = pd.read_excel("/content/drive/MyDrive/sources/data_paristrading.xlsx", index_col=0)
    df.index.name = 'Date'

    df_list = []  # 로그 수익률을 저장할 리스트

    for symbol in df.columns:
        if df.index[0] >= start_train or df.index[-1] <= end_test:
            # 주어진 기간에 해당하지 않는 데이터는 건너뜁니다.
            continue

        log_returns = np.log(df[symbol] / df[symbol].shift(1))  # 로그 수익률 계산
        log_returns = log_returns[start_train:end_train].dropna()  # 훈련 기간 데이터 추출 및 NaN 값 제거
        log_returns.name = symbol  # 컬럼 이름 설정
        df_list.append(log_returns)

    return df_list

# constants 객체를 만들어서 함수에 전달합니다.
constants = Constants("2016-07-01", "2021-07-01", "2021-07-02", "2023-07-02")

df_list = get_dataframes(constants)
# dataframes 리스트에는 각 주식의 로그 수익률이 들어 있습니다.

In [ ]:
cat = pd.concat(df_list,axis=1)
cat = cat.dropna(axis=1, thresh=int(len(cat)*0.75)) # we require that at least 75% of the data is present
cat = cat.bfill(axis=1) # fill nans by backfilling
cat_T = cat.transpose()
cat_T = cat_T.dropna(axis=1)

print("Concatenated dataframe along columns:\n", cat.iloc[:5,:5])
print("\nTaking the transpose:\n", cat_T.iloc[:5,:5])
print("\nDimension of cat_T:", cat_T.shape)

In [ ]:
'''
cat_T.values returns a np.array of log returns
We will standardise these log returns using StandardScaler
'''

norm = StandardScaler().fit_transform(cat_T.values)
norm_symbols = cat_T.index

print("Values before standardisation:\n", cat_T.values[:3])
print("\nValues after standardisation:\n", norm[:3])
print(np.mean(norm).round(1), np.var(norm).round(1))

PCA

In [ ]:
'''
Dimensionality Reduction with PCA before we apply any clustering algos:
'''

def optimise_pca(norm):
    explained_var = []
    principal_components = []
    for i in range(0,733,1): # For loop from 0 to 49 principal components to calculate their respective percentage of variance.

        # Initialise the PCA function with the i number of principal components.
        pca_model = PCA(n_components=i)

        # Fits the model with norm and applies PCA on norm to return dataset with i number of principal components
        pca_model.fit_transform(norm)

        # Using the attribute ".explained_variance_ratio" returns percentage of variance explained by each selected component.
        # This result is then summed up to obtained percentage of variance explained for i number of components.
        # It is appended into explained_var to be plotted as the y-axis.
        explained_var.append(sum(pca_model.explained_variance_ratio_))

        # The number of principal components, i, is appended to principal_components to be plotted as the x-axis.
        principal_components.append(i)

    # Plot graph of "Percentage of variance explained" against "Number of principal components"
    # Using this graph, the optimal number of dimensions can be found via the elbow method
    plt.plot(principal_components, explained_var, color="red", marker="o")
    plt.title("Graph of variance explained against no. of PCs")
    plt.xlabel("Number of principal components")
    plt.ylabel("Percentage of variance explained")
    plt.axvline(270, color='black', linestyle='--')
    plt.show()

def pca(norm, symbols, n):

    # Initialise the PCA function with the n number of principal components.
    pca_model = PCA(n_components=n)

    # Fits the model with norm and applies PCA on norm to return dataset with n number of principal components
    components = pca_model.fit_transform(norm)

    # print(f"{n} largest eigenvalues of the covariance matrix:\n{pca_model.explained_variance_}")
    # print(f"Percentage of variance explained by each component:\n{pca_model.explained_variance_ratio_}")
    print(f"Total percentage variance explained with {n} principal components: {round(sum(pca_model.explained_variance_ratio_),3)}\n")
    print()

    # Create dataset (that has been reduced to n number of principal components) in proper format using pd.Dataframe(data,columns,index):
    pca_df = pd.DataFrame(components, columns=[f"PC{k}" for k in range(1,n+1)],index=symbols)
    return pca_df

optimise_pca(norm)

In [ ]:
pca_df = pca(norm, norm_symbols, n=270)
print(pca_df.iloc[:,:5])

In [ ]:
'''Make a pointer to store clustering results'''

cluster_results = {}

'''K-Means Clustering'''

def optimise_k(pca_df):

    error = []
    ks = []

    for i in range(10, 300, 20): # For loop from 10 to 200 with intervals of 5 and calculate the respective sum of squared distances for i number of centroids

        # Initialise the KMeans function with the i number of centroids.
        # By default, KMeans function uses the KMeans++ Method of initialisation the centroids.
        kmeans = KMeans(n_clusters=i)

        # Fits the model with pcd_df and applies KMeans algorithm on pca_df.
        kmeans.fit(pca_df)

        # Using the attribute ".inertia_" returns the sum of squared distances of datapoints to their closest cluster center.
        # It is appended into error to be plotted as the y-axis.
        error.append(kmeans.inertia_)

        # The number of centroids, i, is appended to ks to be plotted as the x-axis.
        ks.append(i)

    # Plot graph of "Sum of Squared Distances" against "k value".
    # Using this graph, the optimal number of centroids can be found via the elbow method
    plt.plot(ks, error, marker="o", color="r")
    plt.title("Graph of sum of squared distance against k")
    plt.xlabel("k value")
    plt.ylabel("Sum of Squared Distances")
    plt.axvline(150, color='black', linestyle='--')
    plt.show()

def cluster_kmeans(pca_df, k):

    # Initialise the KMeans function with the k number of centroids.
    kmeans = KMeans(n_clusters=k)

    # Fits the model with pcd_df and applies KMeans algorithm on pca_df.
    kmeans.fit(pca_df)

    # Using the method .predict(X), it calculates the closest cluster each datapoint in dataset X belongs to.
    results = kmeans.predict(pca_df)

    # Plot the cluster results.
    fig, ax = plt.subplots(figsize=(9,6))
    pca_df.plot(x="PC1", y="PC2", kind="scatter",
        title=f"K-Means Cluster Results, k={k}", c=results, cmap="jet", ax=ax)
    plt.show()
    return results

optimise_k(pca_df)
print(f"From the graph above, we can see that the optimal k-value is at the elbow, at k = 30")

In [ ]:
'''Plotting the K-Means Cluster Results'''

# K-Means Clustering is applied on pca_df for k = 3
kmeans_results = cluster_kmeans(pca_df,k=150)

# K-Means Clustering results are saved in cluster_results
cluster_results[f"KMeans"] = kmeans_results

print(f"Therefore, we plot the K-Means results for the optimal k-value, 150")

DBSCAN

In [ ]:
'''
DBSCAN Clustering:
'''

def optimise_epsilon(pca_df):

    # Initialise the NearestNeighbors function with 2 neighbors
    # This is to find optimal value of epsilon based on an optimisation algorithm
    neigh = NearestNeighbors(n_neighbors=2)

    # Fits the model with pcd_df
    nbrs = neigh.fit(pca_df)

    # Using the method .kneighbors(X), the k-Neighbours for datatpoints in datatset X are found
    # By default, this method returns the distances array which is found in the 0 index
    distances = nbrs.kneighbors(pca_df)[0]

    # Sort distances row-wise
    distances = np.sort(distances,axis=0)

    # Remove the 0-index value (Euclidean distance to itself which is 0)
    distances = distances[:,1:]

    # Plot graph of "Euclidean Distance" against "Datapoint".
    # Using this graph, the optimal epsilon value can be found via the elbow method
    plt.plot(distances, color="r")
    plt.title("Sorted euclidean distance to nearest neighbour")
    plt.xlabel("Datapoint")
    plt.ylabel("Euclidean Distance")
    plt.axhline(y=50, color='black', linestyle='--')
    plt.show()

def cluster_dbscan(pca_df, eps, min_samples):

    # Initialise the DBSCAN function with eps and min_samples.
    # eps is a float value and is the maximum distance between two samples for one to be considered as in the neighborhood of the other
    # min_samples is the number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
    # This includes the point itself.
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)

    # Using the method .fit_predict(X), clusters from a data or distance matrix and predict labels are computed
    results = dbscan.fit_predict(pca_df)

    # Plot the cluster results.
    fig, ax = plt.subplots(figsize=(9,6))
    pca_df.plot(x="PC1", y="PC2", kind="scatter",
        title=f"DBSCAN Cluster Results", c=results, cmap="jet", ax=ax)
    plt.show()

    # Get number of clusters formed.
    num_clusters = np.max(results)
    print(f"For the epsilon value of {eps}, the number of clusters is {num_clusters}")

    return results

optimise_epsilon(pca_df)
print(f"From the graph above, we can see that the optimal epsilon is at the elbow, at epsilon = 16")

In [ ]:
'''Plotting the DBSCAN Cluster Results'''

# DBSCAN Clustering is applied on pca_df for eps = 50 and min_samples = 2
results = cluster_dbscan(pca_df, eps=50, min_samples=2)

# DBSCANS Clustering results are saved in cluster_results, a dict
cluster_results["DBSCAN"] = results

hierarchical Clustering

In [ ]:
'''
Hierarchical Clustering (AgglomerativeClustering)
'''

def cluster_hierar(pca_df, n):

    # Initialise the AgglomerativeClustering function with n clusters.
    # Fits the model with pca_df and applies Agglomerative Clustering algorithm on pca_df.
    hierar = AgglomerativeClustering(n_clusters=n, ).fit(pca_df)

    # Using the method .fit_predict(X), it fits and return the result of each datapoint's clustering assignment.
    results = hierar.fit_predict(pca_df)

    # Plot cluster results.
    fig, ax = plt.subplots(figsize=(9,6))
    pca_df.plot(x="PC1", y="PC2", kind="scatter",
        title=f"Agglomerative Clustering Results, n={n}", c=results, cmap="jet", ax=ax)
    plt.show()

    return results

def draw_dendrogram(pca_df):

    # Initialise Linkage function with linkage method = "ward".
    # linkage returns a linkage matrix, which is needed for the dendrogram to be plotted.
    linked = linkage(pca_df,method = "ward")

    # Plot dendrogram results.
    plt.figure(figsize =(10,7))
    plt.title("Dendrogram")
    dendrogram(linked,
                orientation = "top",
                distance_sort = "descreasing",
                show_leaf_counts = True)
    plt.show()

draw_dendrogram(pca_df)

In [ ]:
'''Plotting the Hierarchical Cluster Results'''
# Hierarchical Clustering is applied on pca_df for n_clusters = sqrt(N)
results = cluster_hierar(pca_df, int(pca_df.shape[0]**0.5))

# Hierarchical Clustering results are saved in cluster_results
cluster_results["Hierarchical"] = results

print(f"In the case of Hierarchical Clustering, we chose the optimal number of clusters to be sqrt(N)=27, where N is the number of data")

In [ ]:
'''
Cluster Results -> Groupings of stocks/ETFs:
1. First, we need to map the cluster ids to the symbols of our stocks to get groups of stocks
2. Then store the clusters into a pointer -> "groups"
'''

# Return array of the symbols of stocks.
print(norm_symbols.shape)

# Contains cluster group assignments for each algorithm.
print([x.shape for x in cluster_results.values()])

def map_clusters(cluster_result):
    grouping = [[] for _ in range(max(cluster_result)+1)]
    for i, res in enumerate(cluster_result):
        grouping[res].append(norm_symbols[i])

    grouping = [g for g in grouping if 1<len(g)<=10] # Keep clusters with <=10 members.

    return grouping

groups = {}
for algorithm, cluster_result in cluster_results.items():
    groups[algorithm] = map_clusters(cluster_result)


print("Number of valid clusters:", {k:len(v) for k,v in groups.items()})
#print("Example cluster:", )

In [ ]:
groups['DBSCAN']

In [ ]:
groups['KMeans']

In [ ]:
k = pd.DataFrame(groups['KMeans'])
k = k.T
k.to_excel('코스피_전체_Kmeans.xlsx')

In [ ]:
groups['Hierarchical']

In [ ]:
h = pd.DataFrame(groups['Hierarchical'])
h = h.T
h.to_excel('코스피_전체_Hierarchical.xlsx')

In [ ]:
'''
Evaluating cluster results by measuring pairwise correlation within each cluster in the validation period
'''
df = pd.read_excel("/content/drive/MyDrive/sources/data_paristrading.xlsx", index_col=0)
df.index.name = 'Date'
def get_r_squared(df, group, start_test, end_test):
    start_test = dt.datetime.strptime(start_test, "%Y-%m-%d")
    end_test = dt.datetime.strptime(end_test, "%Y-%m-%d")
    dfs = []


    for symbol in df.columns:
        if symbol in group:
            if df.index[0] >= start_test or df.index[-1] <= end_test:
                # 주어진 기간에 해당하지 않는 데이터는 건너뜁니다.
                continue

            log_returns = np.log(df[symbol] / df[symbol].shift(1))  # 로그 수익률 계산
            log_returns = log_returns[start_test:end_test].dropna()  # 훈련 기간 데이터 추출 및 NaN 값 제거
            log_returns.name = symbol  # 컬럼 이름 설정

            dfs.append(df[symbol])


    df = pd.concat(dfs, axis=1)
    df = df[start_test:end_test] # Filter to validation period
    corr = df.corr() # Correlation matrix
    corr.values[np.tril_indices_from(corr)] = np.nan # Keep only upper triangular of the correlation matrix
    return corr.unstack().mean() # Return mean of the correlation matrix

def average_correlation(groups, constants):
    avg_corr = {}
    for algorithm, cluster_result in groups.items():
        corrs = []
        for group in cluster_result:
            corr = get_r_squared(df, group, constants.start_test, constants.end_test)
            corrs.append(corr)
        avg_corr[algorithm] = np.mean(np.square(corrs)) # Take mean of r-squared
    return avg_corr

corr_metrics = average_correlation(groups, constants)
pd.DataFrame(corr_metrics.items(), columns=["Algorithm", "R-Squared"])

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/sources/data_paristrading.xlsx", index_col=0)
df.index.name = 'Date'

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔글꼴 Regular으로 폰트 지정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path, size=20)
# 폰트 설정
plt.rc('font', family='NanumGothic')
matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 기호(-)가 깨지는 현상 방지

In [ ]:
'''
Plot the stock prices of stocks from each cluster in the validation period
'''


def plot_time_series(df, group, i, start_test, end_test):
    start_test = dt.datetime.strptime(start_test, "%Y-%m-%d")
    end_test = dt.datetime.strptime(end_test, "%Y-%m-%d")

    for symbol in df.columns:
        if symbol in group:
            if df.index[0] >= start_test or df.index[-1] <= end_test:
                # 주어진 기간에 해당하지 않는 데이터는 건너뜁니다.
                continue

            log_returns = np.log(df[symbol] / df[symbol].shift(1))  # 로그 수익률 계산
            log_returns = log_returns[start_test:end_test].dropna()  # 훈련 기간 데이터 추출 및 NaN 값 제거
            log_returns.name = symbol  # 컬럼 이름 설정
            df["Cumulative Log Return"] = log_returns.cumsum()
            ax[i].plot(df["Cumulative Log Return"][start_test:end_test]) # Plot validation period
            ax[i].set_xlabel("Datetime")
            ax[i].set_ylabel("Return")
    ax[i].legend(group)

constants = Constants("2016-07-01", "2021-07-01", "2021-07-02", "2023-07-02")
# Plot out clusters.
group = groups["KMeans"]
num_clusters = len(group)

fig, ax = plt.subplots(math.ceil(num_clusters/2),2, figsize = (15,20))
ax = ax.ravel()
for i in range(0, num_clusters):
    plot_time_series(df, group[i], i, constants.start_test, constants.end_test)
plt.show()